In [2]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from dune_client.client import DuneClient
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

2025-08-20 11:17:47,681 INFO matplotlib.font_manager generated new fontManager


In [4]:
load_dotenv()

GECKO_API = os.getenv('GECKO_API')
#DUNE_API = os.getenv('DUNE_API')

coingecko_url = 'https://api.coingecko.com/api/v3' 

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": GECKO_API
}

# Fetch daily price data for Ethereum

In [9]:
import requests

coin_id = "ethereum"
url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": "usd",
    "days": "30"
}
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'prices': [[1753095819134, 3812.4336207173587], [1753099443871, 3817.309851327789], [1753103021363, 3814.8132813034013], [1753106605714, 3817.027710022185], [1753110221213, 3844.649079097578], [1753113665246, 3820.7778025577786], [1753117459570, 3775.790427503677], [1753121002789, 3767.8418038724217], [1753124620994, 3743.824597207203], [1753128165085, 3734.823105955333], [1753131808212, 3759.3077732765564], [1753135457767, 3754.6150388271694], [1753139015892, 3768.2224739894236], [1753142609314, 3764.7889876542345], [1753146251915, 3747.002161279485], [1753149845925, 3792.3844895636516], [1753153342644, 3748.2599436077007], [1753157050837, 3729.5605510410974], [1753160637084, 3680.039303520269], [1753164218931, 3685.514720089148], [1753167825296, 3689.8443397326855], [1753171410116, 3683.4413159883566], [1753175056550, 3648.5232704613086], [1753178641630, 3661.087118364301], [1753182097437, 3686.6836410758247], [1753185907192, 3706.1542624545114], [1753189545532, 3703.3590136006183],

# Fetch Whale Transfers using dune sim

In [10]:
import os
from dune_client.client import DuneClient

DUNE_CUSTOM_API = os.getenv("DUNE_CUSTOM_API")
if not DUNE_CUSTOM_API:
    raise ValueError("Please set DUNE_CUSTOM_API environment variable.")

dune = DuneClient(DUNE_CUSTOM_API)

ValueError: Please set DUNE_CUSTOM_API environment variable.

In [ ]:
# Initialize Dune client
dune = DuneClient("DUNE_CUSTOM_API")  

# Fetch whale transaction data
from dune_client.client import DuneClient
dune = DuneClient(DUNE_CUSTOM_API)
response = dune.get_custom_endpoint_result(
  "mrcryptolov3r",
  "ethereum-whales",
  limit = 100
)


# Convert response to DataFrame
df = pd.DataFrame(response.get_rows())

# Format whale volume column as currency
if 'amount_usd' in df.columns:
    df['amount_usd'] = df['amount_usd'].astype(float).map(lambda x: f"${x:,.2f}")

print(df.head(30))  # Display first 30 rows

In [ ]:
# ### Fetch Ethereum Price Data
# Get daily Ethereum price data for the last 30 days from CoinGecko.
import requests
import pandas as pd

# Define CoinGecko endpoint and parameters
coingecko_url = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart"
params = {
    "vs_currency": "usd",
    "days": "30",
    "interval": "daily"
}

# Fetch price data
cg_response = requests.get(coingecko_url, params=params)
cg_data = cg_response.json()

# Convert to DataFrame
eth_prices = pd.DataFrame(cg_data['prices'], columns=['timestamp', 'eth_price'])
eth_prices['timestamp'] = pd.to_datetime(eth_prices['timestamp'], unit='ms')
eth_prices['date'] = eth_prices['timestamp'].dt.date
eth_prices = eth_prices[['date', 'eth_price']]

In [ ]:
import requests
import pandas as pd

# Example: Fetch Ethereum daily prices for the last 90 days
url = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart"
params = {"vs_currency": "usd", "days": "90", "interval": "daily"}
response = requests.get(url, params=params)
data = response.json()

# Convert to DataFrame
price_df = pd.DataFrame(data['prices'], columns=['timestamp', 'price_usd'])
price_df['timestamp'] = pd.to_datetime(price_df['timestamp'], unit='ms')
price_df['date'] = price_df['timestamp'].dt.date
daily_price_df = price_df.groupby('date')['price_usd'].mean().reset_index()

In [ ]:
# Convert to datetime and keep the full datetime for flexibility
df['block_time'] = pd.to_datetime(df['block_time'])
df['date'] = df['block_time'].dt.date

# Aggregate whale volume by day
if df['amount_usd'].dtype == 'O':
    daily_whale_volume = (
        df.groupby('date')['amount_usd']
        .apply(lambda x: pd.to_numeric(x.str.replace('[$,]', '', regex=True), errors='coerce').sum())
        .reset_index()
    )
else:
    daily_whale_volume = df.groupby('date')['amount_usd'].sum().reset_index()

daily_whale_volume.columns = ['date', 'whale_volume_usd']
print(daily_whale_volume.head(20))

# Merge DataSet

In [ ]:
merged = pd.merge(eth_prices, daily_whale_volume, on='date', how='inner')
# Format columns as currency and round
merged['whale_volume_usd'] = merged['whale_volume_usd'].round(2).map(lambda x: f"${x:,.2f}")
merged['eth_price'] = merged['eth_price'].round(2).map(lambda x: f"${x:,.2f}")

print(merged[['date', 'whale_volume_usd', 'eth_price']].head(30))  # Display first 30 rows

In [ ]:
# Example: Convert your price data into daily format
price_df['date'] = pd.to_datetime(price_df['timestamp']).dt.date  # Adjust column name to match your data
daily_price_df = price_df.groupby('date')['price_usd'].mean().reset_index()  # daily average price

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Example: merge whale volume and price movement
merged_corr = pd.merge(
    daily_whale_volume,  # from your aggregation
    daily_price_df,      # your price movement data
    on='date',
    how='inner'
)

print(merged_corr.head())

# Relationship between price and whale transfer amounts

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.bar(merged_corr['date'], merged_corr['whale_volume_usd'], color='tab:blue', alpha=0.6, label='Whale Volume (USD)')
ax1.set_xlabel('Date')
ax1.set_ylabel('Whale Volume (USD)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax2 = ax1.twinx()
ax2.plot(merged_corr['date'], merged_corr['price_usd'], color='tab:orange', label='ETH Price (USD)', linewidth=2)
ax2.set_ylabel('ETH Price (USD)', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')
fig.tight_layout()
plt.title('Whale Volume (Bar) & ETH Price (Line) - Last 30 Days')
fig.legend(loc='upper left', bbox_to_anchor=(0.1,0.9))
plt.show()

In [ ]:
# ### Correlation Analysis
# Calculate and print the correlation between whale volume and ETH price.
# Remove formatting for correlation calculation
merged_corr = pd.merge(eth_prices, daily_whale_volume, on='date', how='inner')
corr_value = merged_corr['whale_volume_usd'].corr(merged_corr['eth_price'])
print(f"Correlation between whale volume and ETH price (daily, last 30 days): {corr_value:.4f}")

# Correlation between Whale Transfers vs Daily Price Change

In [ ]:
# ### Visualization
# Plot daily whale volume and ETH price with dual y-axes for trend analysis.
import matplotlib.pyplot as plt

# Create plot with dual y-axes
fig, ax1 = plt.subplots(figsize=(12,6))

ax1.set_xlabel('Date')
ax1.set_ylabel('Whale Volume (USD)', color='tab:blue')
ax1.plot(merged_corr['date'], merged_corr['whale_volume_usd'], color='tab:blue', label='Whale Volume (USD)')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('ETH Price (USD)', color='tab:orange')
ax2.plot(merged_corr['date'], merged_corr['eth_price'], color='tab:orange', label='ETH Price (USD)')
ax2.tick_params(axis='y', labelcolor='tab:orange')

fig.tight_layout()
plt.title('Daily Whale Volume vs ETH Price (Last 30 Days)')
plt.show()

# Correlation between Transfer Amount, No. of Transactions, No. of Whales and Price Change (%)

In [ ]:
import seaborn as sns

metrics = combined_dataframe[['amount', 'transactions', 'whales', 'price_change_pct']].dropna()

corr_matrix = metrics.corr()

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()